# Game of 24: ToT + DFS

## Utils

### Imports

In [1]:
from langchain_openai import ChatOpenAI
from environments.game_of_24.evaluate_utils import GameOf24EvaluateOutputParser, game_of_24_evaluate_prompt
from environments.game_of_24.generate_utils import GameOf24GenerateOutputParser, game_of_24_generate_prompt
from planning_library.custom_agent_executor.strategies import TreeOfThoughtsDFSStrategy
from planning_library.custom_agent_executor.strategies.toolllm.thought_generators import AgentThoughtGenerator
from planning_library.custom_agent_executor.strategies.toolllm.thought_evaluators import LLMThoughtEvaluator
import os
import re

%load_ext autoreload
%autoreload 2

### Enabling W&B 

In [2]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

### Defining components

#### Hyperparameters

In [3]:
# ToT hyperparameters
max_num_thoughts = 3  # number of thoughts to generate at each iteration
max_num_steps = 20  # total maximum number of iterations
value_threshold = 0.49  # threshold for evaluation; only thoughts with value > value_threshold will be explored

# other hyperparameters
model_name = "gpt-3.5-turbo"
temperature = 0.8

#### Thought Evaluator

Evaluation prompt is adapted from the [original implementation](https://github.com/princeton-nlp/tree-of-thought-llm/blob/master/src/tot/prompts/game24.py).

In [4]:
game_of_24_evaluate_prompt.input_variables

['thought']

In [5]:
print(game_of_24_evaluate_prompt.format(thought="2 + 3 = 5 (left: 6 5 4)"))

System: You are a helpful assistant that judges whether given numbers can reach 24.
Human: Evaluate if given numbers can reach 24 (sure/likely/impossible)
Human: Input: 2 + 8 = 10 (left: 10 14)
Judge:
AI: 10 + 14 = 24
sure
Human: Input: 2 * 6 = 12 (left: 11 12)
Judge:
AI: 11 + 12 = 23
12 - 11 = 1
11 * 12 = 132
11 / 12 = 0.91
impossible
Human: Input: 2 + 2 = 4 (left: 4 4 10)
Judge:
AI: 4 + 4 + 10 = 8 + 10 = 18
4 * 10 - 4 = 40 - 4 = 36
(10 - 4) * 4 = 6 * 4 = 24
sure
Human: Input: 10 + 1 = 11 (left: 4 9 11)
Judge:
AI: 9 + 11 + 4 = 20 + 4 = 24
sure
Human: Input: 2 * 4 = 8 (left: 5 7 8)
Judge:
AI: 5 + 7 + 8 = 12 + 8 = 20
(8 - 5) * 7 = 3 * 7 = 21
I cannot obtain 24 now, but numbers are within a reasonable range
likely
Human: Input: 2 * 3 = 6 (left: 5 6 6)
Judge:
AI: 5 + 6 + 6 = 17
(6 - 5) * 6 = 1 * 6 = 6
I cannot obtain 24 now, but numbers are within a reasonable range
likely
Human: Input: 2 * 5 = 10 (left: 10 10 11)
Judge:
AI: 10 + 10 + 11 = 31
(11 - 10) * 10 = 10
10 10 10 are all too big
i

In [6]:
evaluator_chain = (
    {
        "inputs": lambda x: x["inputs"] if not x["current_state"] else x["current_state"][-1][1],
        "thought": lambda x: x["thought"].log,
    }
    | game_of_24_evaluate_prompt
    | ChatOpenAI(model=model_name, temperature=temperature)
    | GameOf24EvaluateOutputParser()
)
thought_evaluator = LLMThoughtEvaluator(llm_chain=evaluator_chain)

#### Thought Generator

Generation prompt is adapted from the [original implementation](https://github.com/princeton-nlp/tree-of-thought-llm/blob/master/src/tot/prompts/game24.py).

In [7]:
game_of_24_generate_prompt.input_variables

['inputs', 'max_num_thoughts']

In [8]:
print(game_of_24_generate_prompt.format(inputs="2 2 3 4", max_num_thoughts="3"))

System: You are a helpful assistant that proposes next steps in Game of 24.
Human: Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Each step, you are only allowed to choose two of the remaining numbers to obtain a new number.
Human: Input: 2 8 8 14
8 variants for a possible next step:
AI: 2 + 8 = 10 (left: 8 10 14)
8 / 2 = 4 (left: 4 8 14)
14 + 2 = 16 (left: 8 8 16)
2 * 8 = 16 (left: 8 14 16)
8 - 2 = 6 (left: 6 8 14)
14 - 8 = 6 (left: 2 6 8)
14 /  2 = 7 (left: 7 8 8)
14 - 2 = 12 (left: 8 8 12)
Human: Input: 10 14
1 variants for a possible next step:
AI: 10 + 14 = 24
Human: Input: 2 2 3 4
3 variants for a possible next step:


In [9]:
from typing import Dict, Any


def process_inputs(x: Dict[str, Any]) -> str:
    if not x["intermediate_steps"]:
        return x["inputs"]

    next_numbers = re.search("^.*?\(left: (.*?)\)$", x["intermediate_steps"][-1][1])

    if next_numbers:
        return next_numbers.groups()[0]

    return x["intermediate_steps"][-1][1]


agent = (
    {
        "inputs": lambda x: process_inputs(x),
        "max_num_thoughts": lambda x: x["max_num_thoughts"],
        "agent_scratchpad": lambda x: x["intermediate_steps"],
    }
    | game_of_24_generate_prompt
    | ChatOpenAI(model=model_name, temperature=temperature)
    | GameOf24GenerateOutputParser()
)
thought_generator = AgentThoughtGenerator(thought_generation_mode="propose")

#### Defining a trivial tool

In [10]:
from langchain.tools import tool


@tool
def simple_tool(text: str) -> str:
    """Returns its input."""
    return text

### Defining strategy

In [11]:
strategy_executor = TreeOfThoughtsDFSStrategy(
    thought_generator=thought_generator,
    thought_evaluator=thought_evaluator,
    agent=agent,
    max_num_thoughts=max_num_thoughts,
    max_num_steps=max_num_steps,
    value_threshold=value_threshold,
    tools=[simple_tool],
    verbose=True,
)

# Running strategy

In [13]:
strategy_executor.invoke({"inputs": "5 3 4 5"})



> Entering new TreeOfThoughtsDFSStrategy chain...
5 + 3 = 8 (left: 4 5 8 5)5 + 3 = 8 (left: 4 5 8 5)4 + 5 = 9 (left: 9 8 5)4 + 5 = 9 (left: 9 8 5)9 + 5 = 14 (left: 8 14)9 + 5 = 14 (left: 8 14)8 + 14 = 22 (left: 22)8 + 14 = 22 (left: 22)22 - 2 = 2022 - 2 = 2020 + 2 = 2220 + 2 = 2222 + 20 = 4222 + 20 = 4242 - 20 = 22 (left: 22)42 - 20 = 22 (left: 22)

OutputParserException: GameOf24EvaluateOutputParser expected output value to be one of dict_keys(['sure', 'likely', 'impossible']) (case-insensitive). Received i'm sorry, but given the input 22 - 2 = 20, it is impossible to reach 24..

In [14]:
strategy_executor.invoke({"inputs": "5 3 4 5"})



> Entering new TreeOfThoughtsDFSStrategy chain...
5 + 3 = 8 (left: 4 5 8 5)5 + 3 = 8 (left: 4 5 8 5)4 + 5 = 9 (left: 9 8 5)4 + 5 = 9 (left: 9 8 5)9 + 8 = 17 (left: 5 17)9 + 8 = 17 (left: 5 17)5 + 17 = 22 (left: 22)5 + 17 = 22 (left: 22)22 + 2 = 24 (left: 2)22 + 2 = 24 (left: 2)

> Finished chain.


{'inputs': '5 3 4 5',
 'output': 'Agent stopped due to no promising thoughts available.'}

In [15]:
strategy_executor.invoke({"inputs": "5 3 4 5"})



> Entering new TreeOfThoughtsDFSStrategy chain...
5 + 3 = 8 (left: 4 5 8 5)5 + 3 = 8 (left: 4 5 8 5)

> Finished chain.


{'inputs': '5 3 4 5',
 'output': 'Agent stopped due to no promising thoughts available.'}

In [16]:
strategy_executor.invoke({"inputs": "5 3 4 5"})



> Entering new TreeOfThoughtsDFSStrategy chain...
5 + 4 = 9 (left: 3 5 9 5)5 + 4 = 9 (left: 3 5 9 5)3 + 9 = 12 (left: 5 5 12)3 + 9 = 12 (left: 5 5 12)5 + 5 = 10 (left: 10 12)5 + 5 = 10 (left: 10 12)10 + 12 = 22 (left: 22)10 + 12 = 22 (left: 22)22 + 2 = 24 (left: 2)22 + 2 = 24 (left: 2)2 + 2 = 42 + 2 = 44 + 2 = 64 + 2 = 66 + 4 = 10 (left: 2 10)6 + 4 = 10 (left: 2 10)2 + 10 = 12 (left: 12)2 + 10 = 12 (left: 12)12 + 12 = 24

> Finished chain.


{'inputs': '5 3 4 5', 'output': '12 + 12 = 24'}